<a href="https://colab.research.google.com/github/ran4erep/Stable-Colab/blob/main/Stable_Diffusion_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

↑ It is recommended to click File --> Save a copy on Disk

This is needed to keep all the data you have entered. For example, the token for CivitAI site, so you don't have to copy and paste it into the corresponding form every time.

In [ ]:
# @title Stable Diffusion Installing and connecting to Google Drive: { vertical-output: true, form-width: "10%", display-mode: "form" }
# @markdown Do you wish to use Google Drive?
use_gdrive = False # @param {type:"boolean"}
# @markdown

import os
import re
import torch
import gc
import sys
import shutil
from google.colab import files
from PIL import Image
import io
import numpy as np
from datetime import date
from datetime import datetime
from IPython.display import clear_output
from IPython.display import display, update_display
import ipywidgets as widgets
import requests
from tqdm import tqdm
from urllib.parse import unquote

if use_gdrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  print("GDrive sucessfully mounted to /content/gdrive")
  # Создаём папку в Google Drive, если её нет
  if not os.path.exists("/content/gdrive/MyDrive/SDOutput"):
    os.makedirs("/content/gdrive/MyDrive/SDOutput", exist_ok=True)
    print("Folder successfully created /content/gdrive/MyDrive/SDOutput")

#!pip install diffusers["torch"] transformers
!pip install diffusers
!pip install accelerate
!pip install compel
#!pip install git+https://github.com/huggingface/diffusers
#!pip install compel --upgrade

from diffusers import AutoPipelineForText2Image, AutoPipelineForImage2Image, AutoPipelineForInpainting
from diffusers import (
    DDPMScheduler,
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    DPMSolverMultistepScheduler,
)
from diffusers.utils import make_image_grid, load_image
from compel import Compel, ReturnedEmbeddingsType
import transformers
used_models_array = [""]

previous_checkpoint = None
previous_lora_path = None
pipe = None
previous_mode = None
downloaded_model_name = None
downloaded_model_basename = None
downloaded_lora_name = None
downloaded_lora_basename = None
current_url = None
previous_url = None
lora_url = None
previous_lora_url = None
civitai_models = []
previous_clip_skip = None

clear_output()
print("Installation is complete :)")

In [ ]:
# @title Stable Diffusion preparing: { vertical-output: true, form-width: "10%", display-mode: "form" }
# @markdown
# @markdown Models to be loaded into the script:
# @markdown
# @markdown [CivitAI](https://civitai.com/models)
# @markdown
# @markdown [HuggingFace](https://huggingface.co/models?library=diffusers)

#!nvidia-smi

####################
# Список опробованных мною моделей:
# PicX_real:       GraydientPlatformAPI/picx-real
# Juggernaut:      digiplay/Juggernaut_final
# Juggernaut XL:   frankjoshua/juggernautXL_version6Rundiffusion
# RealVisXL 3.0:   SG161222/RealVisXL_V3.0_Turbo
# Base 1.5:        runwayml/stable-diffusion-v1-5
# Base SDXL:       stabilityai/stable-diffusion-xl-base-1.0
# Base SDXL Turbo: stabilityai/sdxl-turbo
####################


# @markdown ---
# @markdown Your token from the CivitAI website (optional for downloading models from CivitAI that require registration on the website.
# @markdown
# @markdown You can get it in the settings of your CivitAI account, in the API Keys block):
civitai_token = "" # @param {type:"string"}
# @markdown Модель для загрузки:
current_checkpoint = "" # @param ["runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-xl-base-1.0", "runwayml/stable-diffusion-inpainting", "stabilityai/sdxl-turbo", "digiplay/Juggernaut_final", "GraydientPlatformAPI/picx-real", "frankjoshua/juggernautXL_version6Rundiffusion", "SG161222/RealVisXL_V3.0_Turbo", "SG161222/Realistic_Vision_V6.0_B1_noVAE", "stablediffusionapi/lob-realvisxl-v20"] {allow-input: true}
# @markdown Use safetensors?
use_safetensors = True # @param ["True", "False", "None"] {type:"raw"}

# @markdown ---
# @markdown Use LoRA?
use_lora = False # @param {type:"boolean"}
# @markdown Current LoRA:
lora_path = "" # @param ["nerijs/pixel-art-xl", "ntc-ai/SDXL-LoRA-slider.cinematic-lighting"] {allow-input: true}
# @markdown LoRA weights name:
weight_name = "" # @param ["pixel-art-xl.safetensors", "cinematic lighting.safetensors"] {allow-input: true}
# @markdown Link to LoRA from website CivitAI:
lora_url = "" # @param {type:"string"}
# @markdown ---
current_device = "cuda"
# @markdown CLIP skip
# @markdown (1 means that all layers are used, this is the default value):
clip_skip = 1 # @param {type:"slider", min:1, max:12, step:1}
## @markdown Вариант:
#current_variant = "fp16" # @param ["", "fp16", "ema"]

display_handle = display(None, display_id=True)

def load_civit_model(url):
  global current_checkpoint, previous_checkpoint, downloaded_model_name, downloaded_model_basename, previous_url, current_url, civitai_models
  for element in civitai_models:
    if current_checkpoint in element[0]:
      current_checkpoint = element[1]
      return

  if current_url != previous_url:
    if civitai_token:
      url = url + "&token=" + civitai_token
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 100000
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    if 'content-disposition' in response.headers:
      content_disposition = response.headers['content-disposition']
      filename_index = content_disposition.find('filename=')
      if filename_index != -1:
        filename = content_disposition[filename_index + len('filename='):]
        filename = unquote(filename)
        filename = filename.strip('"')
        downloaded_model_name = filename
    if not downloaded_model_name:
      downloaded_model_name = os.path.basename(url)

    downloaded_model_basename, _ = os.path.splitext(downloaded_model_name)
    with open(downloaded_model_name, 'wb') as file:
      for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
      print("Model loading error :(")

    print("\nModel " + downloaded_model_basename + " successfully loaded :) ")
    print("Wait, model conversion is in progress....\n")

    from pkg_resources import get_distribution
    diffusers_version = get_distribution('diffusers').version
    lib_url = "https://raw.githubusercontent.com/huggingface/diffusers/v" + diffusers_version + "/scripts/convert_original_stable_diffusion_to_diffusers.py"
    !wget "$lib_url"

    !python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "$downloaded_model_name" --dump_path "$downloaded_model_basename/" --from_safetensors
    previous_url = current_checkpoint
    current_checkpoint = downloaded_model_basename
    civitai_models.append((url, downloaded_model_basename))
    os.remove(downloaded_model_name)
    os.remove("convert_original_stable_diffusion_to_diffusers.py")
    clear_output()
    print("Model " + downloaded_model_basename + " successfully converted and ready to work :)")

repo_regexp = r'[^\/\s]+\s*\/\s*[^\/\s]+'
link_regexp = r'^https?://\S+$'

if re.match(repo_regexp, current_checkpoint):
  current_checkpoint = current_checkpoint
elif re.match(link_regexp, current_checkpoint):
  current_url = current_checkpoint
  load_civit_model(current_checkpoint)
else:
  print("Could not find the model from your link. :( Please provide a link to the HuggingFace repository of the form \"author/model_name\" or a direct link to the .safetensors file from the CivitAI website.")

def flush():
  del(pipe)
  gc.collect()
  torch.cuda.empty_cache()


if pipe and not use_lora:
  pipe.unload_lora_weights()


def run_chkpt(current_checkpoint, lora_path, use_lora, mode):
    global previous_checkpoint, previous_lora_path, previous_mode, lora_url, previous_lora_url, downloaded_lora_basename, downloaded_lora_name, pipe, clip_skip, previous_clip_skip
    if current_checkpoint != previous_checkpoint or mode != previous_mode or previous_clip_skip != clip_skip:
        used_models_array.append(current_checkpoint)
        print("Installing the model...")
        if clip_skip > 1:
          text_encoder = transformers.CLIPTextModel.from_pretrained(current_checkpoint, subfolder = "text_encoder", num_hidden_layers = 12 - (clip_skip - 1), torch_dtype = torch.float16)
        if mode == "txt2img":
          if clip_skip > 1:
            pipe = AutoPipelineForText2Image.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors, text_encoder = text_encoder)
          else:
            pipe = AutoPipelineForText2Image.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors)
        if mode == "img2img":
          if clip_skip > 1:
            pipe = AutoPipelineForImage2Image.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors, text_encoder = text_encoder)
          else:
            pipe = AutoPipelineForImage2Image.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors)
        if mode == "inpainting":
          if clip_skip > 1:
            pipe = AutoPipelineForInpainting.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors, text_encoder = text_encoder)
          else:
            pipe = AutoPipelineForInpainting.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors)
        pipe = pipe.to(current_device)
        pipe.safety_checker = None
        if current_scheduler == "ddpm":
          pipe.scheduler = ddpm.from_config(pipe.scheduler.config)
        if current_scheduler == "ddim":
          pipe.scheduler = ddim.from_config(pipe.scheduler.config)
        if current_scheduler == "pndm":
          pipe.scheduler = pndm.from_config(pipe.scheduler.config)
        if current_scheduler == "lms":
          pipe.scheduler = lms.from_config(pipe.scheduler.config)
        if current_scheduler == "euler":
          pipe.scheduler = euler.from_config(pipe.scheduler.config)
        if current_scheduler == "euler_anc":
          pipe.scheduler = euler_anc.from_config(pipe.scheduler.config)
        if current_scheduler == "dpm":
          pipe.scheduler = dpm.from_config(pipe.scheduler.config)
        print("Everything is ready to go :)")
        clear_output()
        previous_mode = mode
        previous_checkpoint = current_checkpoint
    if use_lora:
      if lora_path != previous_lora_path and lora_url == "":
        if pipe:
          pipe.unload_lora_weights()
        print("Installing LoRA model...")
        pipe.load_lora_weights(lora_path, weight_name=weight_name)
        print("Everything is ready to go :)")
        previous_lora_path = lora_path
      if lora_url != previous_lora_url and lora_path == "" and weight_name == "":
        if pipe:
          pipe.unload_lora_weights()
        response = requests.get(lora_url, stream=True)
        total_size_in_bytes = int(response.headers.get('content-length', 0))
        block_size = 100000
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

        if 'content-disposition' in response.headers:
          content_disposition = response.headers['content-disposition']
          filename_index = content_disposition.find('filename=')
          if filename_index != -1:
            filename = content_disposition[filename_index + len('filename='):]
            filename = unquote(filename)
            filename = filename.strip('"')
            downloaded_lora_name = filename
        if not downloaded_lora_name:
          downloaded_lora_name = os.path.basename(lora_url)

        downloaded_lora_basename, _ = os.path.splitext(downloaded_lora_name)
        with open(downloaded_lora_name, 'wb') as file:
          for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
        progress_bar.close()
        if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
          print("Model loading error :(")
        pipe.load_lora_weights(downloaded_lora_name)
        previous_lora_url = lora_url
        print("Everything is ready to go :)")


# print("\nList of models used during the session:")
# model_fields = []
# for i, model_name in enumerate(used_models_array, start=1):
#     if model_name:
#       model_field = widgets.Text(
#           value=model_name,
#           description=f'Model {i-1}:',
#           disabled=False,
#           layout=widgets.Layout(width="auto")
#       )
#       model_fields.append(model_field)

# display(widgets.VBox(model_fields))

# if len(used_models_array) == 1:
#   print("So far, the list of models is empty :(")

In [ ]:
# @title Setting up the generation parameters: { form-width: "10%", display-mode: "form" }
# Здесь указываем переменные для пайплайна Stable Diffusion
# Дефолтный промпт для лучшего качества: (8k, ultra realistic, highly detailed, cinematic lighting)

# @markdown Prompt:
prompt = "werewolf, forest, night" # @param {type:"string"}
# @markdown Negative prompt:
negs = "" # @param {type:"string"}
# @markdown Generated image style:
styles = "No style" # @param ["No style", "ran4erep's style", "3D model", "Analog film", "Anime", "Cinematic", "Comic book", "Plasticine", "Digital art", "Fantasy", "Isometric", "Sketch", "Low-poly", "Neonpunk", "Origami", "Photo", "Pixel art", "Texture", "Abstraction", "Abstract Expressionism", "Hyperrealism", "Pop-art", "Renaissance", "Steampunk", "Surrealism", "Futurism", "Syntwave-futurism", "Retro-futurism", "Sci-Fi", "Vaporwave futurism", "Dystopian", "Gothic", "Grunge", "Horror", "Lovecraftian", "Creepy", "Minimalism", "Noir", "Long exposure", "Tilt-Shift", "iPhone photo"]
# @markdown ---

def latents_to_rgb(latents):
    weights = (
        (60, -60, 25, -70),
        (60,  -5, 15, -50),
        (60,  10, -5, -35)
    )

    weights_tensor = torch.t(torch.tensor(weights, dtype=latents.dtype).to(latents.device))
    biases_tensor = torch.tensor((150, 140, 130), dtype=latents.dtype).to(latents.device)
    rgb_tensor = torch.einsum("...lxy,lr -> ...rxy", latents, weights_tensor) + biases_tensor.unsqueeze(-1).unsqueeze(-1)
    image_array = rgb_tensor.clamp(0, 255)[0].byte().cpu().numpy()
    image_array = image_array.transpose(1, 2, 0)  # Change the order of dimensions

    return Image.fromarray(image_array)

def decode_tensors(pipe, step, timestep, callback_kwargs):
  if step % 5 == 0:
    latents = callback_kwargs["latents"]

    image = latents_to_rgb(latents)
    image_resized = image.resize((412, 412), Image.NEAREST)
    update_display(image_resized, display_id=display_handle.display_id)

  return callback_kwargs

if styles != "No style":
  if styles == "ran4erep's style":
    prompt = prompt + ", 8k, ultra realistic, highly detailed"
    negs = negs + ", canvas frame, cartoon, 3d, disfigured, bad art, deformed, extra limbs, close up, b&w, wierd colors, blurry, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, out of frame, ugly, extra limbs, bad anatomy, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, mutated hands, fused fingers, too many fingers, long neck, Photoshop, video game, ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, mutation, mutated, extra limbs, extra legs, extra arms, disfigured, deformed, cross-eye, body out of frame, blurry, bad art, bad anatomy, 3d render"
  if styles == "3D model":
    prompt = "professional 3d model {" + prompt + "} . octane render, highly detailed, volumetric, dramatic lighting"
    negs   = negs + ", ugly, deformed, noisy, low poly, blurry, painting"
  if styles == "Analog film":
    prompt = "analog film photo {" + prompt + "} . faded film, desaturated, 35mm photo, grainy, vignette, vintage, Kodachrome, Lomography, stained, highly detailed, found footage"
    negs   = negs + ", painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured"
  if styles == "Anime":
    prompt = "anime artwork {" + prompt + "} . anime style, key visual, vibrant, studio anime, highly detailed"
    negs   = negs + ", photo, deformed, black and white, realism, disfigured, low contrast"
  if styles == "Cinematic":
    prompt = "cinematic film still {" + prompt + "} . shallow depth of field, vignette, highly detailed, high budget, bokeh, cinemascope, moody, epic, gorgeous, film grain, grainy"
    negs   = negs + ", anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured"
  if styles == "Comic book":
    prompt = "comic {" + prompt + "} . graphic illustration, comic art, graphic novel art, vibrant, highly detailed"
    negs   = negs + ", photograph, deformed, glitch, noisy, realistic, stock photo"
  if styles == "Plasticine":
    prompt = "play-doh style {" + prompt + "} . sculpture, clay art, centered composition, Claymation"
    negs   = negs + ", sloppy, messy, grainy, highly detailed, ultra textured, photo"
  if styles == "Digital art":
    prompt = "concept art {" + prompt + "} . digital artwork, illustrative, painterly, matte painting, highly detailed"
    negs   = negs + ", photo, photorealistic, realism, ugly"
  if styles == "Fantasy":
    prompt = "ethereal fantasy concept art of {" + prompt + "} . magnificent, celestial, ethereal, painterly, epic, majestic, magical, fantasy art, cover art, dreamy"
    negs   = negs + ", photographic, realistic, realism, 35mm film, dslr, cropped, frame, text, deformed, glitch, noise, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, sloppy, duplicate, mutated, black and white"
  if styles == "Isometric":
    prompt = "isometric style {" + prompt + "} . vibrant, beautiful, crisp, detailed, ultra detailed, intricate"
    negs   = negs + ", deformed, mutated, ugly, disfigured, blur, blurry, noise, noisy, realistic, photographic"
  if styles == "Sketch":
    prompt = "line art drawing {" + prompt + "} . professional, sleek, modern, minimalist, graphic, line art, vector graphics"
    negs   = negs + ", anime, photorealistic, 35mm film, deformed, glitch, blurry, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, mutated, realism, realistic, impressionism, expressionism, oil, acrylic"
  if styles == "Low-poly":
    prompt = "low-poly style {" + prompt + "} . low-poly game art, polygon mesh, jagged, blocky, wireframe edges, centered composition"
    negs   = negs + ", noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo"
  if styles == "Neonpunk":
    prompt = "neonpunk style {" + prompt + "} . cyberpunk, vaporwave, neon, vibes, vibrant, stunningly beautiful, crisp, detailed, sleek, ultramodern, magenta highlights, dark purple shadows, high contrast, cinematic, ultra detailed, intricate, professional"
    negs   = negs + ", painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured"
  if styles == "Origami":
    prompt = "origami style {" + prompt + "} . paper art, pleated paper, folded, origami art, pleats, cut and fold, centered composition"
    negs   = negs + ", noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo"
  if styles == "Photo":
    prompt = "cinematic photo {" + prompt + "} . 35mm photograph, film, bokeh, professional, 4k, highly detailed"
    negs   = negs + ", drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly"
  if styles == "Pixel art":
    prompt = "pixel-art {" + prompt + "} . low-res, blocky, pixel art style, 8-bit graphics"
    negs   = negs + ", sloppy, messy, blurry, noisy, highly detailed, ultra textured, photo, realistic"
  if styles == "Texture":
    prompt = "texture {" + prompt + "} top down close-up"
    negs   = negs + ", ugly, deformed, noisy, blurry"
  if styles == "Abstraction":
    prompt = "abstract style {" + prompt + "} . non-representational, colors and shapes, expression of feelings, imaginative, highly detailed"
    negs   = negs + ", realistic, photographic, figurative, concrete"
  if styles == "Abstract Expressionism":
    prompt = "abstract expressionist painting {" + prompt + "} . energetic brushwork, bold colors, abstract forms, expressive, emotional"
    negs   = negs + ", realistic, photorealistic, low contrast, plain, simple, monochrome"
  if styles == "Hyperrealism":
    prompt = "hyperrealistic art {" + prompt + "} . extremely high-resolution details, photographic, realism pushed to extreme, fine texture, incredibly lifelike"
    negs   = negs + ", simplified, abstract, unrealistic, impressionistic, low resolution"
  if styles == "Pop-art":
    prompt = "pop Art style {" + prompt + "} . bright colors, bold outlines, popular culture themes, ironic or kitsch"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, minimalist"
  if styles == "Renaissance":
    prompt = "renaissance style {" + prompt + "} . realistic, perspective, light and shadow, religious or mythological themes, highly detailed"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, modernist, minimalist, abstract"
  if styles == "Steampunk":
    prompt = "steampunk style {" + prompt + "} . antique, mechanical, brass and copper tones, gears, intricate, detailed"
    negs   = negs + ", deformed, glitch, noisy, low contrast, anime, photorealistic"
  if styles == "Surrealism":
    prompt = "surrealist art {" + prompt + "} . dreamlike, mysterious, provocative, symbolic, intricate, detailed"
    negs   = negs + ", anime, photorealistic, realistic, deformed, glitch, noisy, low contrast"
  if styles == "Futurism":
    prompt = "futuristic style {" + prompt + "} . sleek, modern, ultramodern, high tech, detailed"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, vintage, antique"
  if styles == "Syntwave-futurism":
    prompt = "retro cyberpunk {" + prompt + "} . 80’s inspired, synthwave, neon, vibrant, detailed, retro futurism"
    negs   = negs + ", modern, desaturated, black and white, realism, low contrast"
  if styles == "Retro-futurism":
    prompt = "retro-futuristic {" + prompt + "} . vintage sci-fi, 50s and 60s style, atomic age, vibrant, highly detailed"
    negs   = negs + ", contemporary, realistic, rustic, primitive"
  if styles == "Sci-Fi":
    prompt = "sci-fi style {" + prompt + "} . futuristic, technological, alien worlds, space themes, advanced civilizations"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, historical, medieval"
  if styles == "Vaporwave futurism":
    prompt = "vaporwave style {" + prompt + "} . retro aesthetic, cyberpunk, vibrant, neon colors, vintage 80s and 90s style, highly detailed"
    negs   = negs + ", monochrome, muted colors, realism, rustic, minimalist, dark"
  if styles == "Dystopian":
    prompt = "dystopian style {" + prompt + "} . bleak, post-apocalyptic, somber, dramatic, highly detailed"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, cheerful, optimistic, vibrant, colorful"
  if styles == "Gothic":
    prompt = "gothic style {" + prompt + "} . dark, mysterious, haunting, dramatic, ornate, detailed"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, cheerful, optimistic"
  if styles == "Grunge":
    prompt = "grunge style {" + prompt + "} . textured, distressed, vintage, edgy, punk rock vibe, dirty, noisy"
    negs   = negs + ", smooth, clean, minimalist, sleek, modern, photorealistic"
  if styles == "Horror":
    prompt = "horror-themed {" + prompt + "} . eerie, unsettling, dark, spooky, suspenseful, grim, highly detailed"
    negs   = negs + ", cheerful, bright, vibrant, light-hearted, cute"
  if styles == "Lovecraftian":
    prompt = "lovecraftian horror {" + prompt + "} . eldritch, cosmic horror, unknown, mysterious, surreal, highly detailed"
    negs   = negs + ", light-hearted, mundane, familiar, simplistic, realistic"
  if styles == "Creepy":
    prompt = "macabre style {" + prompt + "} . dark, gothic, grim, haunting, highly detailed"
    negs   = negs + ", bright, cheerful, light-hearted, cartoonish, cute"
  if styles == "Minimalism":
    prompt = "minimalist style {" + prompt + "} . simple, clean, uncluttered, modern, elegant"
    negs   = negs + ", ornate, complicated, highly detailed, cluttered, disordered, messy, noisy"
  if styles == "Noir":
    prompt = "film noir style {" + prompt + "} . monochrome, high contrast, dramatic shadows, 1940s style, mysterious, cinematic"
    negs   = negs + ", ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, vibrant, colorful"
  if styles == "Long exposure":
    prompt = "long exposure photo of {" + prompt + "} . Blurred motion, streaks of light, surreal, dreamy, ghosting effect, highly detailed"
    negs   = negs + ", static, noisy, deformed, shaky, abrupt, flat, low contrast"
  if styles == "Tilt-Shift":
    prompt = "tilt-shift photo of {" + prompt + "} . selective focus, miniature effect, blurred background, highly detailed, vibrant, perspective control"
    negs   = negs + ", blurry, noisy, deformed, flat, low contrast, unrealistic, oversaturated, underexposed"
  if styles == "iPhone photo":
    prompt = "iphone photo {" + prompt + "} . large depth of field, deep depth of field, highly detailed"
    negs   = negs + ", drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly, shallow depth of field, bokeh"
# @markdown For which Stable Diffusion version you have loaded the model? (It's important for workability tokens weight, otherwise there will be an error)
sd_type = "Stable Diffusion 1.5/2.0" # @param ["Stable Diffusion 1.5/2.0", "Stable Diffusion XL (Turbo)"]
# @markdown Token weight for SD 1.5 is written as + or - after the word inside the token: "cat playing with ball+++++", "cat+ playing with ball"
# @markdown
# @markdown The weight for SDXL is written as a fractional number after brackets with a word or even a whole token inside it: "cat (playing with ball)1.5"
# @markdown
# @markdown ---
# @markdown Width:
width = 512 # @param {type:"number"}
# @markdown Height:
height = 512 # @param {type:"number"}
# @markdown Number of images:
images_count = 1 # @param {type:"slider", min:1, max:10, step:1}
# @markdown Steps:
steps = 25 # @param {type:"number"}
# @markdown Guidance scale:
gscale = 7.5 # @param {type:"number"}
# @markdown Current scheduler:
current_scheduler = "euler" # @param ["ddpm", "ddim", "pndm", "lms", "euler_anc", "euler", "dpm"] {type:"string"}

# @markdown Use random seed?
randomness = True # @param {type:"boolean"}
# @markdown Seed:
seed = 5107167905980673 # @param {type:"integer"}

### Run Stable Diffusion
Do not run this block of code in its entirety. Expand it and you will see the different modes of Stable Diffusion. Run the mode you need.

In [ ]:
# @title Text2Image: { form-width: "10%", display-mode: "form" }
# @markdown
# @markdown <-- Start generation
# Генерация картинки
# @markdown
# @markdown Show diffusion in real time?
show_diffusion = True # @param {type:"boolean"}

ddpm = DDPMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
ddim = DDIMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
pndm = PNDMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
lms = LMSDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler_anc = EulerAncestralDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler = EulerDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
dpm = DPMSolverMultistepScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")

run_chkpt(current_checkpoint, lora_path, use_lora, "txt2img")

if sd_type == "Stable Diffusion 1.5/2.0":
  compel_proc = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
  prompt_embeds = compel_proc(prompt)

if sd_type == "Stable Diffusion XL (Turbo)":
  compel = Compel(
    tokenizer=[pipe.tokenizer, pipe.tokenizer_2] ,
    text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
    returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
    requires_pooled=[False, True]
  )
  conditioning, pooled = compel(prompt)

#####################################################################
# Информация о текущей генерации
output = [
    "# Current Stable Diffusion model: " + current_checkpoint,
]
if use_lora:
    if lora_url == "" and lora_path and weight_name:
      output.append("# Current LoRA model: " + lora_path)
    elif lora_url and lora_path == "" and weight_name == "":
      output.append("# Current LoRA model: " + downloaded_lora_basename)
output.extend([
    "# Current style: " + styles,
    "# Prompt: " + prompt,
    "# Negative prompt: " + negs,
    "# Resolution: " + str(width) + "x" + str(height),
    "# Number of generated images: " + str(images_count)
])
max_length_string = max(output, key=len)

print("#" * len(max_length_string) )
for item in output:
  print(item)

######################################################################
if not randomness:
  generator = torch.Generator(current_device).manual_seed(seed)
  print("# Current generation's seed: " + str(seed) + "\n" + ("#" * len(max_length_string)) )
elif randomness:
  current_seed = torch.Generator(current_device).seed()
  generator = torch.Generator(current_device).manual_seed(current_seed)
  print("# Current generation's seed: " + str(current_seed) + "\n" + ("#" * len(max_length_string) ))


#result = pipe(prompt=prompt, height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)

if sd_type == "Stable Diffusion 1.5/2.0":
  if show_diffusion:
    torch.cuda.empty_cache()
    result = pipe(prompt_embeds=prompt_embeds, callback_on_step_end=decode_tensors, callback_on_step_end_tensor_inputs=["latents"], height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)
  else:
    torch.cuda.empty_cache()
    result = pipe(prompt_embeds=prompt_embeds, height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)

if sd_type == "Stable Diffusion XL (Turbo)":
  if show_diffusion:
    torch.cuda.empty_cache()
    result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, callback_on_step_end=decode_tensors, callback_on_step_end_tensor_inputs=["latents"], height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)
  else:
    torch.cuda.empty_cache()
    result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)

# Вывод и сохрарнение результата

if use_gdrive:
  today = str(date.today())
  if not os.path.exists("/content/gdrive/MyDrive/SDOutput/" + today):
    os.makedirs("/content/gdrive/MyDrive/SDOutput/" + today, exist_ok=True)
  for i in range(images_count):
    time = datetime.now()
    time = str(time.strftime("%H-%M-%S"))
    file_name = time + "_" + "seed_" + str(current_seed) + "_image" + str(i) +".png"
    file_name = "/content/gdrive/MyDrive/SDOutput/" + today + "/" + file_name
    images = make_image_grid(result.images, rows=1, cols=images_count)
    result.images[i].save(file_name)
    print("Saved as " + file_name)

clear_output()
#update_display(result, display_id=display_handle.display_id)
print("#" * len(max_length_string) )
print("# Current Stable Diffusion model: " + current_checkpoint)
if use_lora:
  if lora_url == "" and lora_path and weight_name:
    output.append("# Current LoRA model: " + lora_path)
  elif lora_url and lora_path == "" and weight_name == "":
    output.append("# Current LoRA model: " + downloaded_lora_basename)
print("# Current style: " + styles)
print("# Prompt: " + prompt)
print("# Negative prompt: " + negs)
print("# Resolution: " + str(width) + "x" + str(height))
print("# Number of generated images: " + str(images_count))
if not randomness:
  print("# Current generation's seed: " + str(seed) + "\n" + ("#" * len(max_length_string) ))
elif randomness:
  print("# Current generation's seed: " + str(current_seed) + "\n" + ("#" * len(max_length_string) ))
make_image_grid(result.images, rows=1, cols=images_count)

In [ ]:
# @title Image loading for Image2Image module
print("Pick source image: ")
init_image = files.upload()
image_key = list(init_image.keys())[0]
image_data = init_image[image_key]
img = Image.open(io.BytesIO(image_data))
img_width, img_height = img.size

In [ ]:
# @title Image2Image:
# @markdown <-- Start generation

# @markdown ---
# @markdown Strength:

# @markdown The lower the value, the closer the generation will be to the image you uploaded. The higher the value, the stronger the neural network will be creative.

img2img_strength = 0.3 # @param {type:"slider", min:0.1, max:1.0, step:0.1}

# @markdown Strength must be combined with steps. The total number of steps for noise generation is calculated by the formula (Steps * Strength).

# @markdown Steps:
steps = 25 # @param {type:"number"}
# @markdown ---
# @markdown Show diffusion in real time?
show_diffusion = True # @param {type:"boolean"}
# @markdown ---

ddpm = DDPMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
ddim = DDIMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
pndm = PNDMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
lms = LMSDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler_anc = EulerAncestralDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler = EulerDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
dpm = DPMSolverMultistepScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")

run_chkpt(current_checkpoint, lora_path, use_lora, "img2img")


if init_image:

  if sd_type == "Stable Diffusion 1.5/2.0":
    compel_proc = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
    prompt_embeds = compel_proc(prompt)

  if sd_type == "Stable Diffusion XL (Turbo)":
    compel = Compel(
      tokenizer=[pipe.tokenizer, pipe.tokenizer_2] ,
      text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
      returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
      requires_pooled=[False, True]
    )
    conditioning, pooled = compel(prompt)

  output = [
      "# Current Stable Diffusion model: " + current_checkpoint,
  ]
  if lora_url == "" and lora_path and weight_name:
    output.append("# Current LoRA model: " + lora_path)
  elif lora_url and lora_path == "" and weight_name == "":
    output.append("# Current LoRA model: " + downloaded_lora_basename)
  output.extend([
      "# Current style: " + styles,
      "# Prompt: " + prompt,
      "# Negative prompt: " + negs,
      "# Resolution: " + str(img_width) + "x" + str(img_height),
      "# Strength: " + str(img2img_strength)
  ])
  max_length_string = max(output, key=len)

  print("#" * len(max_length_string) )
  for item in output:
    print(item)

  if not randomness:
    generator = torch.Generator(current_device).manual_seed(seed)
    print("# Current generation's seed: " + str(seed) + "\n" + ("#" * len(max_length_string)) )
  elif randomness:
    current_seed = torch.Generator(current_device).seed()
    generator = torch.Generator(current_device).manual_seed(current_seed)
    print("# Current generation's seed: " + str(current_seed) + "\n" + ("#" * len(max_length_string) ))

  if sd_type == "Stable Diffusion 1.5/2.0":
    if show_diffusion:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=prompt_embeds, callback_on_step_end=decode_tensors, callback_on_step_end_tensor_inputs=["latents"], image=img, strength = img2img_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]
    else:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=prompt_embeds, image=img, strength = img2img_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]

  if sd_type == "Stable Diffusion XL (Turbo)":
    if show_diffusion:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, callback_on_step_end=decode_tensors, callback_on_step_end_tensor_inputs=["latents"], image=img, strength = img2img_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]
    else:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, image=img, strength = img2img_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]

  clear_output()
  print("#" * len(max_length_string) )
  print("# Current Stable Diffusion model: " + current_checkpoint)
  if use_lora:
    if lora_url == "" and lora_path and weight_name:
      output.append("# Current LoRA model: " + lora_path)
    elif lora_url and lora_path == "" and weight_name == "":
      output.append("# Current LoRA model: " + downloaded_lora_basename)
  print("# Current style: " + styles)
  print("# Prompt: " + prompt)
  print("# Negative prompt: " + negs)
  print("# Resolution: " + str(img_width) + "x" + str(img_height))
  print("# Strength: " + str(img2img_strength))
  if not randomness:
    print("# Current generation's seed: " + str(seed) + "\n" + ("#" * len(max_length_string) ))
  elif randomness:
    print("# Current generation's seed: " + str(current_seed) + "\n" + ("#" * len(max_length_string) ))

  if use_gdrive:
    today = str(date.today())
    if not os.path.exists("/content/gdrive/MyDrive/SDOutput/" + today):
      os.makedirs("/content/gdrive/MyDrive/SDOutput/" + today, exist_ok=True)
    time = datetime.now()
    time = str(time.strftime("%H-%M-%S"))
    file_name = time + "_" + "seed_" + str(current_seed) + "_image" + "_img2imged.png"
    file_name = "/content/gdrive/MyDrive/SDOutput/" + today + "/" + file_name
    result.save(file_name)
    print("Saved as " + file_name)

  display( make_image_grid([img, result], rows=1, cols=2) )
else:
  print("There is no image to generate from :(")

In [ ]:
# @title Image loading for Inpainting module
print("Pick source image: ")
inpaint_image = files.upload()
print("Pick mask image: ")
inpaint_mask =  files.upload()

inpaint_image_key = list(inpaint_image.keys())[0]
inpaint_mask_image_key = list(inpaint_mask.keys())[0]
inpaint_image_data = inpaint_image[inpaint_image_key]
inpaint_mask_image_data = inpaint_mask[inpaint_mask_image_key]
inpaint_image = Image.open(io.BytesIO(inpaint_image_data)).convert('RGB')
inpaint_mask_image = Image.open(io.BytesIO(inpaint_mask_image_data)).convert('RGB')
inpaint_image_width, inpaint_image_height = inpaint_image.size
inpaint_image_mask_width, inpaint_image_mask_height = inpaint_mask_image.size

In [ ]:
# @title Inpainting:
# @markdown <-- Start generation

# @markdown ---
# @markdown Strength:

# @markdown The lower the value, the closer the generation will be to the image you uploaded. The higher the value, the stronger the neural network will be creative.

inpaint_strength = 1 # @param {type:"slider", min:0.1, max:1.0, step:0.1}

# @markdown Strength must be combined with steps. The total number of steps for noise generation is calculated by the formula (Steps * Strength).

# @markdown Steps:
steps = 25 # @param {type:"number"}
# @markdown ---
# @markdown Show diffusion in real time?
show_diffusion = True # @param {type:"boolean"}
# @markdown ---
# @markdown When drawing a mask, remember that black areas are ignored by the neural network, while white areas are not. Blurred white areas in masks is also supported.

ddpm = DDPMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
ddim = DDIMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
pndm = PNDMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
lms = LMSDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler_anc = EulerAncestralDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler = EulerDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
dpm = DPMSolverMultistepScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")

run_chkpt(current_checkpoint, lora_path, use_lora, "inpainting")

if inpaint_image and inpaint_mask:

  if sd_type == "Stable Diffusion 1.5/2.0":
    compel_proc = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
    prompt_embeds = compel_proc(prompt)

  if sd_type == "Stable Diffusion XL (Turbo)":
    compel = Compel(
      tokenizer=[pipe.tokenizer, pipe.tokenizer_2] ,
      text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
      returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
      requires_pooled=[False, True]
    )
    conditioning, pooled = compel(prompt)

  output = [
      "# Current Stable Diffusion model: " + current_checkpoint,
  ]
  if lora_url == "" and lora_path and weight_name:
    output.append("# Current LoRA model: " + lora_path)
  elif lora_url and lora_path == "" and weight_name == "":
    output.append("# Current LoRA model: " + downloaded_lora_basename)
  output.extend([
      "# Current style: " + styles,
      "# Prompt: " + prompt,
      "# Negative prompt: " + negs,
      "# Source image resolution: " + str(inpaint_image_width) + "x" + str(inpaint_image_height),
      "# Mask image resolution: " + str(inpaint_image_mask_width) + "x" + str(inpaint_image_mask_height),
      "# Strength: " + str(inpaint_strength)
  ])
  max_length_string = max(output, key=len)

  print("#" * len(max_length_string) )
  for item in output:
    print(item)

  if not randomness:
    generator = torch.Generator(current_device).manual_seed(seed)
    print("# Current generation's seed: " + str(seed) + "\n" + ("#" * len(max_length_string)) )
  elif randomness:
    current_seed = torch.Generator(current_device).seed()
    generator = torch.Generator(current_device).manual_seed(current_seed)
    print("# Current generation's seed: " + str(current_seed) + "\n" + ("#" * len(max_length_string) ))

  if sd_type == "Stable Diffusion 1.5/2.0":
    if show_diffusion:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=prompt_embeds, callback_on_step_end=decode_tensors, callback_on_step_end_tensor_inputs=["latents"], image=inpaint_image, mask_image=inpaint_mask_image, width=inpaint_image_width, height=inpaint_image_height, strength = inpaint_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]
    else:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=prompt_embeds, image=inpaint_image, mask_image=inpaint_mask_image, width=inpaint_image_width, height=inpaint_image_height, strength = inpaint_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]

  if sd_type == "Stable Diffusion XL (Turbo)":
    if show_diffusion:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, callback_on_step_end=decode_tensors, callback_on_step_end_tensor_inputs=["latents"], image=inpaint_image, mask_image=inpaint_mask_image, width=inpaint_image_width, height=inpaint_image_height, strength = inpaint_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]
    else:
      torch.cuda.empty_cache()
      result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, image=inpaint_image, mask_image=inpaint_mask_image, width=inpaint_image_width, height=inpaint_image_height, strength = inpaint_strength, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator).images[0]

  clear_output()
  print("#" * len(max_length_string) )
  print("# Current Stable Diffusion model: " + current_checkpoint)
  if use_lora:
    if lora_url == "" and lora_path and weight_name:
      output.append("# Current LoRA model: " + lora_path)
    elif lora_url and lora_path == "" and weight_name == "":
      output.append("# Current LoRA model: " + downloaded_lora_basename)
  print("# Current style: " + styles)
  print("# Prompt: " + prompt)
  print("# Negative prompt: " + negs)
  print("# Source image resolution: " + str(inpaint_image_width) + "x" + str(inpaint_image_height) )
  print("# Mask image resolution: " + str(inpaint_image_mask_width) + "x" + str(inpaint_image_mask_height) )
  print("# Strength: " + str(inpaint_strength))
  if not randomness:
    print("# Current generation's seed: " + str(seed) + "\n" + ("#" * len(max_length_string) ))
  elif randomness:
    print("# Current generation's seed: " + str(current_seed) + "\n" + ("#" * len(max_length_string) ))

  if use_gdrive:
    today = str(date.today())
    if not os.path.exists("/content/gdrive/MyDrive/SDOutput/" + today):
      os.makedirs("/content/gdrive/MyDrive/SDOutput/" + today, exist_ok=True)
    time = datetime.now()
    time = str(time.strftime("%H-%M-%S"))
    file_name = time + "_" + "seed_" + str(current_seed) + "_image" + "_inpainted.png"
    file_name = "/content/gdrive/MyDrive/SDOutput/" + today + "/" + file_name
    result.save(file_name)
    print("Saved as " + file_name)

  display( make_image_grid([inpaint_image, inpaint_mask_image, result], rows=1, cols=3) )
else:
  print("You have canceled the image uploading :(")